In [4]:
#pip install transformers==4.35.0 sentence-transformers==2.2.2 chromadb==0.4.6 accelerate tqdm 

# Creación de un Entorno de Python

En este documento, vamos a explicar cómo configurar un nuevo entorno de Python utilizando tanto `venv` como `conda`.

## Usando venv

`venv` es una herramienta que viene preinstalada con Python (versión 3.3 o superior) y permite crear entornos virtuales de Python aislados.

### Pasos

1. **Crear un entorno virtual:** Abre la terminal y ejecuta el siguiente comando en el directorio donde desees crear el entorno virtual.

   ```bash
   python3 -m venv nombre_del_entorno


2. **Activar el entorno virtual:** Utiliza el siguiente comando para activar el entorno.


  Linux/Mac:

```bash
    source nombre_del_entorno/bin/activate
```


   Windows:

```bash
    .\nombre_del_entorno\Scripts\activate
```


3. **Instalar dependencias desde requirements.txt con pip:** Una vez activado el entorno, puedes instalar todas las dependencias necesarias desde un archivo `requirements.txt` utilizando:

   ```bash
   pip install -r requirements.txt

4. **Desactivar el entorno virtual:** Para salir del entorno virtual, simplemente ejecuta:

```bash
   deactivate

## Usando Conda
conda es un sistema de gestión de paquetes y de entorno que puede instalar paquetes de diferentes lenguajes.

### Pasos
1. **Instalar Conda:** Puedes descargar e instalar Conda desde [este enlace](https://www.anaconda.com/download).

2. **Crear un entorno Conda:** Para crear un nuevo entorno con Conda, abre la terminal y ejecuta:

```bash
    conda create --name nombre_del_entorno python=3.9 ipykernel
```

Para esta biblioteca, vamos a usar python 3.9 y para interactuar con Jupyter Notebooks vamos a usar "ipykernel" o "jupyter"

3. **Activar el entorno Conda:** Utiliza el siguiente comando para activar el entorno.

```bash
    conda activate nombre_del_entorno
```
4. **Instalar dependencias desde requirements.txt con pip:** Al igual que con venv, puedes instalar las dependencias necesarias desde un archivo requirements.txt en el entorno Conda activado utilizando:
```bash
    pip install -r requirements.txt
```

5. Desactivar el entorno Conda: Para salir del entorno Conda, simplemente ejecuta:

```bash
    conda deactivate
```

In [6]:
# 1️⃣ Importar librerías
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import os
import numpy as np
import uuid

# Modelo de embeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(model_name)

def get_embeddings(text):
    return model.encode(text).tolist()

# Inicializar ChromaDB con persistencia
client = chromadb.PersistentClient(path="./chroma_store")
collection = client.get_or_create_collection(name="documentos_ia")

def cargar_documentos():
    docs_path = "docs/"
    documentos = []
    metadatos = []
    ids = []

    for filename in os.listdir(docs_path):
        if filename.endswith(".txt"):
            with open(os.path.join(docs_path, filename), 'r', encoding="utf-8") as f:
                text = f.read().strip()
                chunks = [text[i:i+500] for i in range(0, len(text), 500)]
                
                for i, chunk in enumerate(chunks):
                    documentos.append(chunk)
                    metadatos.append({"source": filename})
                    ids.append(str(uuid.uuid4()))  # ID único
    
    return documentos, metadatos, ids

def crear_base_datos():
    docs, metas, ids = cargar_documentos()
    embeddings = [get_embeddings(d) for d in docs]
    collection.add(documents=docs, embeddings=embeddings, metadatas=metas, ids=ids)
    print("✅ Base de datos creada con éxito.")

# --- CRUD FUNCIONES ---

def create_example(new_doc):
    """Añadir un documento nuevo"""
    embedding = get_embeddings(new_doc)
    doc_id = str(uuid.uuid4())
    metadata = {"source": "nuevo_documento.txt"}
    
    collection.add(
        documents=[new_doc],
        embeddings=[embedding],
        metadatas=[metadata],
        ids=[doc_id]
    )
    print(f"✅ Documento añadido con ID: {doc_id}")

def read_example(query):
    """Consultar documentos similares"""
    query_emb = get_embeddings(query)
    results = collection.query(
        query_embeddings=[query_emb],
        n_results=1
    )
    
    if results["documents"]:
        print(f"📄 Mejor coincidencia: {results['documents'][0][0]}")
        print(f"📁 Fuente: {results['metadatas'][0][0]['source']}")
        print(f"🆔 ID: {results['ids'][0][0]}")
    else:
        print("⚠️ No se encontraron resultados.")

def get_documents_by_id(doc_id):
    """Obtener un documento por ID"""
    results = collection.get(ids=[doc_id], include=["documents", "metadatas", "embeddings"])
    
    if results["documents"]:
        print(f"📄 Documento: {results['documents'][0]}")
        print(f"📁 Fuente: {results['metadatas'][0]['source']}")
    else:
        print("⚠️ Documento no encontrado.")

def update_example(old_id, new_text):
    """Actualizar un documento (eliminar y reinsertar)"""
    collection.delete(ids=[old_id])
    
    new_id = str(uuid.uuid4())
    embedding = get_embeddings(new_text)
    metadata = {"source": "documento_actualizado.txt"}
    
    collection.add(
        documents=[new_text],
        embeddings=[embedding],
        metadatas=[metadata],
        ids=[new_id]
    )
    print(f"♻️ Documento actualizado. Nuevo ID: {new_id}")

def delete_example(doc_id):
    """Eliminar por ID"""
    collection.delete(ids=[doc_id])
    print(f"🗑️ Documento con ID '{doc_id}' eliminado.")

print('listo')

listo


In [7]:
crear_base_datos()

✅ Base de datos creada con éxito.


In [8]:
get_embeddings("Reynaldo González es un ingeniero de software con 10 años de experiencia en desarrollo web y móvil.")

[-0.09879042953252792,
 0.01190008595585823,
 -0.07934799790382385,
 -0.07173143327236176,
 0.04849899187684059,
 -0.08828750252723694,
 0.04442284256219864,
 0.0687875896692276,
 -0.04471668228507042,
 -0.01907440647482872,
 0.009390895254909992,
 0.055638302117586136,
 0.020385688170790672,
 0.0058576553128659725,
 0.05711749941110611,
 0.03409787267446518,
 -0.03325342759490013,
 0.008881219662725925,
 0.02236260659992695,
 -0.06751406937837601,
 0.11720024049282074,
 -0.02992628701031208,
 0.0018127544317394495,
 -0.007679791189730167,
 -0.014972801320254803,
 0.0011125053279101849,
 0.00690567446872592,
 -0.016392158344388008,
 -0.025348830968141556,
 -0.09709928184747696,
 0.034836966544389725,
 0.06626579910516739,
 0.07979563623666763,
 0.050580110400915146,
 -0.02488500252366066,
 -0.004187237471342087,
 0.02115776762366295,
 -0.058077335357666016,
 -0.0793750211596489,
 0.012050851248204708,
 -0.14230816066265106,
 0.029957246035337448,
 -0.003748527728021145,
 -0.07423306256

In [9]:
results = collection.get(include=["embeddings", "documents", "metadatas"])

In [10]:
read_example("Que se alza sobre buenos aires")

📄 Mejor coincidencia: El sol se alzaba sobre las colinas de Buenos Aires, pintando el cielo con tonos naranja y rosa. El aroma del café recién hecho flotaba en el aire mientras la ciudad comenzaba a despertar. Los porteños se preparaban para un nuevo día lleno de actividad, desde los que se dirigían a sus trabajos en el centro hasta los que disfrutaban de un paseo matutino por los parques.

En el corazón de la ciudad, el Obelisco se erguía como un testigo silencioso de la historia argentina. A sus pies, la vida bulli
📁 Fuente: doc1.txt
🆔 ID: c907f58e-53e3-4faf-8910-7da78ab81d25


In [27]:
for id, content, metadata, embedding in zip(results["ids"], results["documents"], results["metadatas"], results["embeddings"]):
    print(f"ID: {id}, Contenido: {content[:100]} ", f"Metadata: {metadata}", f"Embedding: {embedding[:5]}...")

ID: 771abf37-d48e-4201-a56c-8ed4891f0d87, Contenido: El sol se alzaba sobre las colinas de Buenos Aires, pintando el cielo con tonos naranja y rosa. El a  Metadata: {'source': 'doc1.txt'} Embedding: [ 0.05201019 -0.04184364  0.03102785  0.03516812  0.00390169]...
ID: 899f51d4-481d-4b96-a001-cd16fb6919d4, Contenido: ciosa continuaba, con el tráfico constante y el murmullo de las conversaciones. Los turistas se mara  Metadata: {'source': 'doc1.txt'} Embedding: [ 0.06561411  0.0027415   0.01852138 -0.00384714 -0.08065526]...
ID: d70fa0b5-3fe2-4d09-bee8-68a24adf8d9f, Contenido:  los restaurantes servían deliciosas parrilladas.

Hacia el norte, el elegante barrio de Recoleta in  Metadata: {'source': 'doc1.txt'} Embedding: [ 0.06329252 -0.05760498  0.01019357  0.01427126 -0.11836303]...
ID: 32882197-f7c8-447e-a17d-c97dc291eb52, Contenido: y novelas exploran la realidad, la fantasía y la condición humana con una prosa exquisita.

El fútbo  Metadata: {'source': 'doc1.txt'} Embedding: [ 0.10750

In [11]:
while True:
    opcion = input("Elige una opción o escribe ayuda: ")
    
    if opcion=="ayuda":
        print("\n--- Menú CRUD ---")
        print("1. Crear documento")
        print("2. Consultar por tema")
        print("3. Actualizar documento (por ID)")
        print("4. Eliminar documento (por ID)")
        print("5. Buscar documento (por ID)")
        print("6. Salir")

    if opcion == "1":
        nuevo_doc = input("Ingresa el texto a añadir: ")
        create_example(nuevo_doc)
        
    elif opcion == "2":
        consulta = input("Consulta: ")
        read_example(consulta)
        
    elif opcion == "3":
        doc_id = input("ID del documento a actualizar: ")
        nuevo_texto = input("Nuevo contenido: ")
        update_example(doc_id, nuevo_texto)
        
    elif opcion == "4":
        doc_id = input("ID del documento a eliminar: ")
        delete_example(doc_id)  

    elif opcion == "5":
        doc_id = input("ID del documento a buscar: ")
        get_documents_by_id(doc_id)
        
    elif opcion == "6":
        print("👋 Saliendo del programa.")
        break


--- Menú CRUD ---
1. Crear documento
2. Consultar por tema
3. Actualizar documento (por ID)
4. Eliminar documento (por ID)
5. Buscar documento (por ID)
6. Salir

--- Menú CRUD ---
1. Crear documento
2. Consultar por tema
3. Actualizar documento (por ID)
4. Eliminar documento (por ID)
5. Buscar documento (por ID)
6. Salir
✅ Documento añadido con ID: 37824e71-9cd9-4968-8c39-e4bfa85da92f
📄 Documento: Hola mundo me llamo Fran 
📁 Fuente: nuevo_documento.txt
👋 Saliendo del programa.


## ¿Cómo sigue esto?

### de forma conceptual debemos:
1. Recuperar el contexto relevante con una busqueda como read_example(query) o similar.
2. Formular el prompt para el llm, juntando la consulta original con los embeddings relevantes.
3. Enviar la Petición al Endpoint (en este caso LM Studio).
4. Recibir y mostrar la respuesta.

In [ ]:
# Esta sección es para la integración con LM Studio y no es parte del CRUD. Es solo un ejemplo de cómo se podría hacer una consulta a un modelo de lenguaje como Llama 3 o similar.
# 2️⃣ Integración con LM Studio (Llama 3 o similar)
import requests
import json

LM_STUDIO_URL = "http://192.168.0.3:1234/v1/chat/completions"  

def query_llama(prompt):
    """Envia un prompt a LM Studio y recibe la respuesta."""
    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 256, 
        # Es posible agregar otros parámetros de generación aca (temperature, top_p, etc.)
    }
    try:
        response = requests.post(LM_STUDIO_URL, headers=headers, data=json.dumps(data))
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except requests.exceptions.RequestException as e:
        print(f"Error al conectar con LM Studio: {e}")
        return None
    except (KeyError, json.JSONDecodeError) as e:
        print(f"Error al procesar la respuesta de LM Studio: {e}")
        return None

def responder_pregunta(pregunta):
    """Busca documentos relevantes y luego consulta a Llama."""
    query_emb = get_embeddings(pregunta)
    results = collection.query(
        query_embeddings=[query_emb],
        n_results=3  # Obtener algunos documentos relevantes como contexto
    )

    if results["documents"]:
        contexto = "\n\n".join(results["documents"][0])
        prompt = f"Basándote en la siguiente información:\n\n{contexto}\n\nResponde a la siguiente pregunta: {pregunta}" # Juntar el contexto obtenido y la pregunta
        respuesta = query_llama(prompt)
        return respuesta
    else:
        return "No se encontraron documentos relevantes para responder a tu pregunta."

# --- Ejemplo de uso ---
if __name__ == "__main__":
    crear_base_datos() 

    while True:
        pregunta_usuario = input("Pregúntame algo sobre los documentos (o escribe 'salir'): ")
        if pregunta_usuario.lower() == 'salir':
            break

        respuesta_ia = responder_pregunta(pregunta_usuario)
        if respuesta_ia:
            print(f"🤖 Respuesta de la IA: {respuesta_ia}")

✅ Base de datos creada con éxito.
🤖 Respuesta de la IA: El sol pinta el cielo con tonos naranja y rosa en Buenos Aires.
Error al conectar con LM Studio: 404 Client Error: Not Found for url: http://192.168.0.3:1234/v1/chat/completions
Error al conectar con LM Studio: 404 Client Error: Not Found for url: http://192.168.0.3:1234/v1/chat/completions
Error al conectar con LM Studio: 404 Client Error: Not Found for url: http://192.168.0.3:1234/v1/chat/completions
Error al conectar con LM Studio: 404 Client Error: Not Found for url: http://192.168.0.3:1234/v1/chat/completions
Error al conectar con LM Studio: 404 Client Error: Not Found for url: http://192.168.0.3:1234/v1/chat/completions
Error al conectar con LM Studio: 404 Client Error: Not Found for url: http://192.168.0.3:1234/v1/chat/completions
Error al conectar con LM Studio: 404 Client Error: Not Found for url: http://192.168.0.3:1234/v1/chat/completions
Error al conectar con LM Studio: 404 Client Error: Not Found for url: http://192.1